In [1]:
import matplotlib.pyplot as plt;
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier , AdaBoostClassifier
from sklearn import tree
from sklearn.ensemble import VotingClassifier
from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import accuracy_score

In [2]:
trainSize = 400000
testSize = 50000
size = trainSize+testSize # 400000 for taining and 50000 for testing
data = pd.read_csv("mixed.csv", nrows=size)
data = data.drop(columns="No.")
columns_titles = ["Time", "Source", "Destination", "Length", "Info", "Protocol"]
data = data.reindex(columns=columns_titles)
label_names = np.unique(data.iloc[:,5])

In [3]:
le = preprocessing.LabelEncoder()
data.iloc[:,1] = le.fit_transform(data.iloc[:,1])
data.iloc[:,2] = le.fit_transform(data.iloc[:,2])
data.iloc[:,4] = le.fit_transform(data.iloc[:,4])
data.iloc[:,5] = le.fit_transform(data.iloc[:,5])

In [4]:
def chooseObsMan(dic_f):
    chosenIDs=np.array(())
    for k,d in dic_f.items():
        temp=np.where(y_train==k)[0]
        if temp.size < dic_f[k]:
            temp = dic_f[k]
        z=np.random.choice(temp,dic_f[k],replace=False)
        chosenIDs=np.concatenate((chosenIDs,z))
    return np.int64(chosenIDs)
def label_index(row,threshold=0):
    
    if threshold==0:
        threshold=np.max(row)
    temp_i=np.where(row>=threshold)[0]
    if temp_i.shape[0]>0:
        return temp_i[0]+1
    else:
        return -1

In [5]:
y_train = data.iloc[:trainSize,5]
x_train = data.iloc[:trainSize,:-1]
#x_train = preprocessing.normalize(data.iloc[:trainSize,:-1])

y_test = data.iloc[trainSize:,5]
x_test = data.iloc[trainSize:,:-1]
#x_test = preprocessing.normalize(data.iloc[trainSize:,:-1])

In [6]:
li = {}
r = int(size/100000 + 1)*30
for i in np.unique(data.iloc[:,-1]):
    if data['Protocol'].value_counts()[i] < r:
        li.update({i : data['Protocol'].value_counts()[i]})    
    else :
        li.update({i : r})

labelled=np.zeros(x_train.shape[0])
labelled[:]=-1
labelledIndex=chooseObsMan(li)
labelled[labelledIndex]=y_train[labelledIndex]

In [7]:
#---------------------------------------------OurModel----------------------------------------
clfs ={}

#clf1=GaussianNB()
#clf2=KNeighborsClassifier(n_jobs=-1)
clf3=RandomForestClassifier ( n_estimators = 20 , random_state = 0 ,n_jobs=-1)
#clf4=SVC(kernel='rbf', probability=True)
clf5=tree.DecisionTreeClassifier()
clf7=AdaBoostClassifier()
#clf8=QuadraticDiscriminantAnalysis()

clfs1=[('AdaBoost', clf7),('j48', clf5), ('RFC', clf3)]
ensemble_models = VotingClassifier(estimators=clfs1, voting='soft')

threshold=0.4
leftNumber=-1
temp_left_Group=np.arange(x_train.shape[0])
for i in range(0,x_train.shape[0]):
    temp_left=sum(labelled==-1)
    #print(i, temp_left)
    if temp_left<=0:
        break
    #ensemble_models = VotingClassifier(estimators=clfs1, voting='soft')
    ensemble_models.fit(x_train[labelled!=-1],y_train[labelled!=-1])
    predicted=ensemble_models.predict_proba(x_train[labelled==-1])
    final_decition=np.apply_along_axis(label_index, 1, predicted,threshold)
    labelled[labelled==-1]=final_decition
    if temp_left==leftNumber:
        threshold-0.01
    threshold_step=0.01
    leftNumber=temp_left
    temp_left=sum(labelled==-1)
    temp_left_Group[i]=temp_left
    #print(temp_left, threshold, (final_decition != -1).mean() *400000 , (predicted != -1).mean() *400000)
    if i >0:
        change_rate=temp_left_Group[i-1] - temp_left_Group[i]
        #print(change_rate)
        if(change_rate<1000):
            threshold=threshold-threshold_step

0 397078
6301 0.4 393652.63248026837 400000.0
1 6301
133 0.4 391556.8957308364 400000.0
6168
2 133
71 0.4 186466.16541353383 400000.0
62
3 71
70 0.39 5633.802816901409 400000.0
1
4 70
67 0.38 17142.85714285714 400000.0
3
5 67
67 0.37 0.0 400000.0
0
6 67
65 0.36 11940.298507462687 400000.0
2
7 65
18 0.35 289230.76923076925 400000.0
47
8 18
0 0.33999999999999997 400000.0 400000.0
18
9 0


In [8]:
print("------------------------------------Our Module---------------------------------------")
print('threshold step=',threshold_step)

print('\n*********Training*********\n')

pre=labelled[labelled!=-1]
org=y_train[labelled!=-1]
pop=metrics.precision_recall_fscore_support(org, pre)
results = pd.DataFrame()
for i in range(0,len(pop)):
    if len(pop[0]) > np.unique(data.iloc[:,5]).shape[0]:
        temp = pd.DataFrame([pop[i][:-1]],columns=label_names)
    else :
        temp = pd.DataFrame([pop[i]],columns=label_names)
    
    results=results.append(temp, ignore_index=True)
#print(len(clfs1),'Classifier')


results.insert(0,'  ',['Precision','Recall','Fscore','Support'])
print(results)
resultsDataFrame1 = results
correctLabelled=metrics.accuracy_score(org, pre,normalize=False)
incorrectLabelled=sum(results.iloc[3,1:])-correctLabelled
print([correctLabelled,incorrectLabelled])
numbers=results.iloc[3,1:]
correctLabelledForEachclass=results.iloc[3,1:] *results.iloc[1,1:]
incorrectLabelledForEachclass=results.iloc[3,1:]-correctLabelledForEachclass
results1 = pd.DataFrame(columns = ['Total','Correct', 'Incorrect'])
results1['Total'] = numbers
results1['Correct'] = correctLabelledForEachclass
results1['Incorrect'] = incorrectLabelledForEachclass
print(results1)


------------------------------------Our Module---------------------------------------
threshold step= 0.01

*********Training*********

                      ARP      BROWSER       DHCP      DHCPv6          DNS  \
0  Precision     1.000000     0.027681   0.032588    1.000000     0.455927   
1     Recall     0.030580     0.126891   1.000000    0.455927     0.017878   
2     Fscore     0.059345     0.045448   0.063120    0.626305     0.034408   
3    Support  5363.000000  1190.000000  35.000000  329.000000  8390.000000   

            FTP     FTP-DATA          HTTP   HTTP/XML  ...    NBNS    NBSS  \
0      0.018031     0.005238      0.092822   0.002653  ...     1.0     1.0   
1      0.005174     0.093516      0.004477   1.000000  ...     1.0     1.0   
2      0.008041     0.009920      0.008541   0.005291  ...     1.0     1.0   
3  28991.000000  1604.000000  33507.000000  91.000000  ...  1442.0  1110.0   

   NTP  OCSP  PKIX-CRL          POP          SMB        SMTP            TCP  \
0  